In [59]:
import matplotlib.pyplot as plt
import requests
import json
from scipy import stats
import pandas as pd
from config import api_key
from pprint import pprint
import yfinance as yf

In [56]:
master_dict = {
    'UNH':{
        'Dates':[],
        'Revenue':[],
        'Stock Price':[]},
    'HUM':{
        'Dates':[],
        'Revenue':[],
        'Stock Price':[]},
    'ANTM':{
        'Dates':[],
        'Revenue':[],
        'Stock Price':[]},
    'CI':{
        'Dates':[],
        'Revenue':[],
        'Stock Price':[]},
    'CNC':{
        'Dates':[],
        'Revenue':[],
        'Stock Price':[]},
    
    }

companies = ['UNH','HUM','ANTM','CI','CNC']

for company in companies:
    url = f"https://financialmodelingprep.com/api/v3/income-statement/{company}\
?period=quarter&limit=5&apikey={api_key}"
    try:
        response = requests.get(url).json()
    except:
        print ("Sorry, we don't have this ticker: " + company)
    for i in range(0,5):
            master_dict[company]['Dates'].append(response[i]['date'])
            master_dict[company]['Revenue'].append(response[i]['revenue'])

pprint (master_dict)

{'ANTM': {'Dates': ['2020-09-30',
                    '2020-06-30',
                    '2020-03-31',
                    '2019-12-31',
                    '2019-09-30'],
          'Revenue': [31158000000,
                      29264000000,
                      29621000000,
                      27407000000,
                      26674000000],
          'Stock Price': []},
 'CI': {'Dates': ['2020-09-30',
                  '2020-06-30',
                  '2020-03-31',
                  '2019-12-31',
                  '2019-09-30'],
        'Revenue': [40955000000,
                    39042000000,
                    38050000000,
                    37890000000,
                    38207000000],
        'Stock Price': []},
 'CNC': {'Dates': ['2020-09-30',
                   '2020-06-30',
                   '2020-03-31',
                   '2019-12-31',
                   '2019-09-30'],
         'Revenue': [29090000000,
                     27712000000,
                     26025000000,


In [57]:
tickerdf = yf.download(tickers = companies,period = '2y',group_by = 'ticker',auto_adjust = True)

cncdf = tickerdf['CNC'].reset_index(drop=False)
unhdf = tickerdf['UNH'].reset_index(drop=False)
humdf = tickerdf['HUM'].reset_index(drop=False)
antmdf = tickerdf['ANTM'].reset_index(drop=False)
cidf = tickerdf['CI'].reset_index(drop=False)

dates = ['9/30/2019','12/31/2019','3/31/2020','6/30/2020','9/30/2020']

cncdf = cncdf.loc[cncdf['Date'].isin(dates)]
unhdf = unhdf.loc[unhdf['Date'].isin(dates)]
humdf = humdf.loc[humdf['Date'].isin(dates)]
antmdf = antmdf.loc[antmdf['Date'].isin(dates)]
cidf = cidf.loc[cidf['Date'].isin(dates)]

def add_stock(df,company):
    for item in list(df['Open']):
        master_dict[company]['Stock Price'].append(item)

add_stock(cncdf,'CNC')
add_stock(unhdf,'UNH')
add_stock(humdf,'HUM')
add_stock(antmdf,'ANTM')
add_stock(cidf,'CI')


[*********************100%***********************]  5 of 5 completed


In [58]:
pprint (master_dict)

{'ANTM': {'Dates': ['2020-09-30',
                    '2020-06-30',
                    '2020-03-31',
                    '2019-12-31',
                    '2019-09-30'],
          'Revenue': [31158000000,
                      29264000000,
                      29621000000,
                      27407000000,
                      26674000000],
          'Stock Price': [233.41120102488708,
                          301.2401464632118,
                          233.01528956326297,
                          258.0653216617353,
                          258.7567761911259]},
 'CI': {'Dates': ['2020-09-30',
                  '2020-06-30',
                  '2020-03-31',
                  '2019-12-31',
                  '2019-09-30'],
        'Revenue': [40955000000,
                    39042000000,
                    38050000000,
                    37890000000,
                    38207000000],
        'Stock Price': [150.4992049002725,
                        202.83850983541697,
          

In [3]:
x = weather_data['temp']
y = weather_data['lat']
scatter = plt.scatter(x,y)
slope, intercept, rvalue, pvalue, stderr = stats.linregress(weather_data['temp'],weather_data['lat'])
regress_values = weather_data['temp'] * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
scatter = plt.plot(weather_data['temp'],regress_values,"r-")
scatter = plt.annotate(line_eq,(15,45),fontsize=15,color="red")
plt.xlabel("temp")
plt.ylabel("lat")

NameError: name 'weather_data' is not defined